
![shope](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRaj2DHAhvHURM-op2wofx_g8rPWpvn8CCrgw&usqp=CAU)

1. <center><h1 style ="background-color:blue">Import some backge</h1></center>

In [ ]:
import wandb
import os
import cv2
from PIL import Image
import string
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import albumentations as alb
#from albumentations.augmentations.  CenterCrop, MedianBlur, VerticalFlip, InvertImg
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import pos_tag, ne_chunk
from textblob import TextBlob
#import textfeatures
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from wordcloud import WordCloud, ImageColorGenerator
from wordcloud import STOPWORDS as stopwords_wc


1. <center><h1 style ="background-color:red">make function show _value_on_bars</h1></center>

In [ ]:
def show_values_on_bars(axs, h_v="v", space=0.4):
    '''Plots the value at the end of the a seaborn barplot.
    axs: the ax of the plot
    h_v: weather or not the barplot is vertical/ horizontal'''
    
    def _show_on_single_plot(ax):
        if h_v == "v":
            for p in ax.patches:
                _x = p.get_x() + p.get_width() / 2
                _y = p.get_y() + p.get_height()
                value = int(p.get_height())
                ax.text(_x, _y, format(value, ','), ha="center") 
        elif h_v == "h":
            for p in ax.patches:
                _x = p.get_x() + p.get_width() + float(space)
                _y = p.get_y() + p.get_height()
                value = int(p.get_width())
                ax.text(_x, _y, format(value, ','), ha="left")

    if isinstance(axs, np.ndarray):
        for idx, ax in np.ndenumerate(axs):
            _show_on_single_plot(ax)
    else:
        _show_on_single_plot(axs)


1. <center><h1 style ="background-color:blue">Show a duplicated image</h1></center>

In [ ]:
train_df = pd.read_csv("../input/shopee-product-matching/train.csv")
image_count = train_df["image"].value_counts().reset_index()
image_count.columns = ["image", "count"]
image_count_duplicates = image_count[image_count["count"] > 1]
print("Total no. of images with duplicates: {:,}".format(len(image_count_duplicates)))
my_colors = ["#EDAC54", "#F4C5B7", "#DD7555", "#B95F18", "#475A20"]

#Plot
fig, ax = plt.subplots(figsize=(16, 7))
plt.bar(x=image_count_duplicates.iloc[::16]["image"],
        height=image_count_duplicates.iloc[::16]["count"],
        color=my_colors[4])
plt.title("Duplicated Images: How many apparitions?", fontsize=20)
plt.xticks([])
plt.xlabel("Image ID", fontsize=16)
plt.ylabel("Count", fontsize=16);

1. <center><h1 style ="background-color:blue">make another function for showing image and information</h1></center>

In [ ]:
def get_image_info(name):
    '''Displays a photo of the image and information on it.
    name: a string containing the code of the image (.jpg format)'''
    
    # Read in the image & corresponding metadata
    sample_image = cv2.imread(train_base + "/" + name)
    sample_image = cv2.cvtColor(sample_image, cv2.COLOR_BGR2RGB)
    sample_df = train_df[train_df["image"] == name]
    
    print(color.BOLD + "Apparitions for this image:" + color.END, len(sample_df), "\n" +
          color.BOLD + "Some titles:" + color.END, sample_df["title"].value_counts().index[:5].values, "\n" +
          color.BOLD + "No. of unique groups:" + color.END, sample_df["label_group"].value_counts().shape[0])

    # Plot image
    plt.figure(figsize=(16, 7))
    plt.imshow(sample_image)
    plt.axis("off")
    plt.show();

In [ ]:
class color:
    BOLD = '\033[1m' + '\033[93m'
    END = '\033[0m'

1. <center><h1 style ="background-color:blue">get data train</h1></center>

In [ ]:
train_base = "../input/shopee-product-matching/train_images"

1. <center><h1 style ="background-color:blue">Some of example to show this codes whtat do mean</h1></center>
 

In [ ]:
sample_name = image_count_duplicates.iloc[10]["image"]
get_image_info(name = sample_name)

In [ ]:
sample_name = image_count_duplicates.iloc[110]["image"]
get_image_info(name = sample_name)

In [ ]:
sample_name = image_count_duplicates.iloc[100]["image"]
get_image_info(name = sample_name)

In [ ]:
sample_name = image_count_duplicates.iloc[111]["image"]
get_image_info(name = sample_name)

In [ ]:
sample_name = image_count_duplicates.iloc[5]["image"]
get_image_info(name = sample_name)

In [ ]:
sample_name = image_count_duplicates.iloc[500]["image"]
get_image_info(name = sample_name)

In [ ]:
sample_name = image_count_duplicates.iloc[501]["image"]
get_image_info(name = sample_name)

In [ ]:
sample_name = image_count_duplicates.iloc[502]["image"]
get_image_info(name = sample_name)

In [ ]:
sample_name = image_count_duplicates.iloc[0]["image"]
get_image_info(name = sample_name)

# Show some of product 

In [ ]:
sample_name = image_count_duplicates.iloc[11]["image"]
get_image_info(name = sample_name)

Another product

In [ ]:
sample_name = image_count_duplicates.iloc[12]["image"]
get_image_info(name = sample_name)

In [ ]:
sample_name = image_count_duplicates.iloc[14]["image"]
get_image_info(name = sample_name)

In [ ]:
sample_name = image_count_duplicates.iloc[122]["image"]
get_image_info(name = sample_name)

In [ ]:
sample_name = image_count_duplicates.iloc[1111]["image"]
get_image_info(name = sample_name)

In [ ]:
sample_name = image_count_duplicates.iloc[1130]["image"]
get_image_info(name = sample_name)

make a clean some of daublicate

In [ ]:
def clean_duplicates(df, train=True):
    '''Intakes the original dataframe and returns it wo duplicated images.
    Converts label_group to string as well.'''
    
    if train == True:
        df["label_group"] = df["label_group"].astype(str)
    df = df.drop_duplicates(subset=['image']).reset_index(drop=True)
    
    return df

In [ ]:
train_df = clean_duplicates(df=train_df)

print("Is train metadata now the same length as the image train folder?", "\n",
      train_df.shape[0] == len(os.listdir(train_base)))

make a lable of group

In [ ]:
# Get count of values on each group
groups_df = train_df["label_group"].value_counts().reset_index()
groups_df.columns = ["group", "count"]

# Print info
print("No. of unique groups: {:,}".format(len(groups_df)), "\n" +
      "Max no. of apparitions in 1 group: {}".format(groups_df["count"].max()), "\n" +
      "Min no. of apparitions in 1 group: {}".format(groups_df["count"].min()))

fig, ax = plt.subplots(figsize=(16, 7))
plt.bar(x=groups_df.iloc[::15]["group"],
        height=groups_df.iloc[::15]["count"],
        color=my_colors[3])
plt.title("Group Count Distribution", fontsize=20)
plt.xticks([])
plt.xlabel("Group ID", fontsize=16)
plt.ylabel("Count", fontsize=16)
plt.show();

In [ ]:
def get_group_info(group_name):
    '''This function shows a sample of 6 images within a group.
    group_name: a string representing the desired group code'''
    
    # Retrieve a sample of 6 images from this group
    sample_names = train_df[train_df["label_group"] == group_name]["image"].\
                    sample(6, random_state=24).values
    sample_text = train_df[train_df["label_group"] == group_name]["title"].\
                    sample(1, random_state=1).values

    # Plot
    fig = plt.figure(figsize=(16, 8))
    plt.suptitle(f"Group: {sample_group}", fontsize=20)
    plt.title(f"{sample_text}", fontsize=15)
    plt.axis("off")
    for k, name in enumerate(sample_names):
        image = cv2.imread(train_base + "/" + name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        fig.add_subplot(2, 3, k+1)
        plt.imshow(image)
        plt.axis("off")
    
    plt.show();

first Example

In [ ]:
sample_group = groups_df["group"][199]
get_group_info(group_name=sample_group)

# Example 2

In [ ]:

sample_group = groups_df["group"][91]
get_group_info(group_name=sample_group)

more example

In [ ]:
sample_group = groups_df["group"][200]
get_group_info(group_name=sample_group)

In [ ]:
sample_group = groups_df["group"][20]
get_group_info(group_name=sample_group)

In [ ]:
sample_group = groups_df["group"][2]
get_group_info(group_name=sample_group)

In [ ]:
sample_group = groups_df["group"][30]
get_group_info(group_name=sample_group)

In [ ]:
sample_group = groups_df["group"][40]
get_group_info(group_name=sample_group)

In [ ]:
data=pd.read_csv('../input/shopee-product-matching/sample_submission.csv')

In [ ]:
data[['posting_id','matches']].to_csv('submission.csv',index=False)
print("Submission Ready :)")